In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf, pandas as pd

In [2]:
df = cudf.read_parquet('../../data/train_data/test.parquet')
df['hour'] = df.ts % (60*60*24)
df['day'] = df.ts % (60*60*24*7)
df['aid2'] = df.aid
print( df.shape )
df.head()

(7683577, 7)


,session,aid,ts,type,hour,day,aid2
0,11098528,11830,1661119200,0,79200,338400,11830
1,11098529,1105029,1661119200,0,79200,338400,1105029
2,11098530,264500,1661119200,0,79200,338400,264500
3,11098530,264500,1661119288,0,79288,338488,264500
4,11098530,409236,1661119369,0,79369,338569,409236


In [3]:
user_features = df.groupby('session').agg({'type':'mean','aid':'count','aid2':'nunique','hour':'mean','day':'mean'})
user_features.columns = ['buy_ratio2','count_item2','unique_item2','hour_mean2','day_mean2']

In [4]:
user_features.head()

,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2
session,,,,,
11098528,0.000000,1,1,79200.000000,338400.000000
11098529,0.000000,1,1,79200.000000,338400.000000
11098530,0.166667,6,2,79665.833333,338865.833333
11098531,0.333333,24,11,79429.916667,338629.916667
11098532,0.000000,2,2,79598.500000,338798.500000


In [5]:
user_features2 = df.groupby('session').agg({'type':'std','hour':'std','day':'std'}).fillna(-1)
user_features2.columns = ['buy_ratio_std2','hour_std2','day_std2']

f32 = ['buy_ratio_std2','hour_std2','day_std2']
for c in f32: user_features2[c] = user_features2[c].astype('float32')

In [6]:
user_features2.head()

,buy_ratio_std2,hour_std2,day_std2
session,,,
12150265,0.000000,1774.130859,123962.179688
12720718,-1.000000,-1.000000,-1.000000
12829669,-1.000000,-1.000000,-1.000000
11939634,0.316228,19785.080078,46515.359375
11959102,0.000000,1505.282471,1505.282471


In [7]:
user_features['repeat2'] = user_features.count_item2 / user_features.unique_item2

f32 = ['buy_ratio2','hour_mean2','day_mean2','repeat2']
for c in f32: user_features[c] = user_features[c].astype('float32')
    
i32 = ['count_item2','unique_item2']
for c in i32: user_features[c] = user_features[c].astype('int32')

In [8]:
user_features = cudf.concat([user_features,user_features2],axis=1)

In [9]:
user_features.head()

,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2
session,,,,,,,,,
11098528,0.000000,1,1,79200.000000,338400.00000,1.000000,-1.000000,-1.000000,-1.000000
11098529,0.000000,1,1,79200.000000,338400.00000,1.000000,-1.000000,-1.000000,-1.000000
11098530,0.166667,6,2,79665.835938,338865.84375,3.000000,0.408248,547.326355,547.326355
11098531,0.333333,24,11,79429.914062,338629.90625,2.181818,0.761387,174.236954,174.236954
11098532,0.000000,2,2,79598.500000,338798.50000,1.000000,0.000000,562.149902,562.149902


In [10]:
user_features.dtypes

buy_ratio2        float32
count_item2         int32
unique_item2        int32
hour_mean2        float32
day_mean2         float32
repeat2           float32
buy_ratio_std2    float32
hour_std2         float32
day_std2          float32
dtype: object

In [11]:
user_features.to_parquet('../../data/item_user_features/user4.pqt')